<a href="https://colab.research.google.com/github/dt2229/sandp500/blob/main/SP500%20lstm_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd /content/drive/MyDrive/DSGA1018 Project

In [ ]:
#DATA_DIR="/content/drive/MyDrive"
#File_PATH = os.path.join(DATA_DIR, "SP500.csv")

#sp500 = pd.read_csv(File_PATH)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import time

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import yfinance

sp500 = yfinance.download (tickers = "^GSPC", start = "2012-01-03", 
                              end = "2022-10-15", interval = "1d")

[*********************100%***********************]  1 of 1 completed


In [ ]:
sp500.reset_index(inplace = True)
sp500['Date'] = sp500['Date'].apply(lambda x: str(x)[:10])
sp500.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,1258.859985,1284.619995,1258.859985,1277.060059,1277.060059,3943710000
1,2012-01-04,1277.030029,1278.729980,1268.099976,1277.300049,1277.300049,3592580000
2,2012-01-05,1277.300049,1283.050049,1265.260010,1281.060059,1281.060059,4315950000
3,2012-01-06,1280.930054,1281.839966,1273.339966,1277.810059,1277.810059,3656830000
4,2012-01-09,1277.829956,1281.989990,1274.550049,1280.699951,1280.699951,3371600000


In [ ]:
sp500.sort_values(by = ['Date'], inplace = True)

In [ ]:
sp500.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [ ]:
sp500['ret'] = sp500['Adj Close'].pct_change()

In [ ]:
data = sp500[['Date', 'Adj Close']]
data.head()

,Date,Adj Close
0,2012-01-03,1277.060059
1,2012-01-04,1277.300049
2,2012-01-05,1281.060059
3,2012-01-06,1277.810059
4,2012-01-09,1280.699951


In [ ]:
#data.columns = ['ds', 'y']
#train = data[data['ds']<'2022']
#test = data[data['ds']>='2022']

In [ ]:
data.set_index('Date', inplace = True)

In [ ]:
import math

In [ ]:
close_prices = data['Adj Close']
values = close_prices.values
training_data_len = math.ceil(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
training_data_len

2172

In [ ]:

test_data = scaled_data[training_data_len-60: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=3)
print(tscv)

mape_lst = []

for train_index, valid_index in tscv.split(x_train):
  X_train, X_valid = x_train[train_index], x_train[valid_index]
  Y_train, Y_valid = y_train[train_index], y_train[valid_index]
  model = keras.Sequential()
  model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(layers.LSTM(100, return_sequences=False))
  model.add(layers.Dense(25))
  model.add(layers.Dense(1))
  model.summary()
  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(X_train, Y_train, batch_size= 1, epochs=3)
  predictions = model.predict(X_valid)
  #predictions = scaler.inverse_transform(predictions)
  metrics = mape(predictions, Y_valid)
  print(metrics)
  mape_lst.append(metrics)


TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 100)           40800     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 25)                2525      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
528/528 [==============================] - 25s 41ms/step - loss: 2.2634e-04
Epoch 2/3
528/528 [=====

In [ ]:
mape_lst

[11.115516870091698, 22.784973922212835, 15.168846639994221]

In [ ]:

model = keras.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 100)           40800     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_6 (Dense)             (None, 25)                2525      
                                                                 
 dense_7 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=3)

Epoch 1/3
2112/2112 [==============================] - 69s 31ms/step - loss: 6.2428e-04
Epoch 2/3
2112/2112 [==============================] - 65s 31ms/step - loss: 1.9552e-04
Epoch 3/3
2112/2112 [==============================] - 66s 31ms/step - loss: 1.9666e-04


In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

17/17 [==============================] - 1s 29ms/step


In [ ]:
mape(predictions, y_test)

11.09966947290352

# 新段落

# 新段落